# Transformer-based Approach for Semantic Job Description & Resume Matching

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

> **The individual task is devoted to data analysis and model verification (literature review, visits to laboratories, etc. are not relevant at this stage of your thesis!)**

# 1. Preprocessing

1. Cleaning Text Data
2. Lowercasing
3. Tokenization
4. Removing Stop Words
5. Stemming/Lemmatization
6. Handling Missing Data

In [2]:
# Libraries

import nltk
import zipfile
import os
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


**Jobs Data**

In [3]:
# Read jobs data

import pandas as pd

jobs_data = pd.read_csv('/kaggle/input/data-analyst-job-postings-google-search/gsearch_jobs.csv')

In [4]:
jobs_data

Unnamed: 0  index                                              title  \
0               0      0                                       Data Analyst   
1               1      1       Entry Level - Business Data Analyst (Remote)   
2               2      2                            Data Analyst/Researcher   
3               3      3  Data analyst/AI expert to help build a website...   
4               4      4                                       Data Analyst   
...           ...    ...                                                ...   
39709       39709    282                     Marketing Data & BI Analyst II   
39710       39710    283                                  Lead-Data Analyst   
39711       39711    284                                  Lead-Data Analyst   
39712       39712    285                                  Lead-Data Analyst   
39713       39713    286  Institutional Credit Management - Lending Data...   

                           company_name            location  \
0                          Cisco Meraki           Anywhere    
1                      Midwest Staffing           Anywhere    
2                            Amyx, Inc.    United States      
3                                Upwork           Anywhere    
4      Bank Street College of Education    United States      
...                                 ...                 ...   
39709                      EDWARD JONES       Houstonia, MO   
39710                      EDWARD JONES      Marshfield, MO   
39711                      EDWARD JONES      High Point, MO   
39712                      EDWARD JONES         Calhoun, MO   
39713                              Citi       United States   

                                                     via  \
0                                           via LinkedIn   
1                                           via LinkedIn   
2                                             via Indeed   
3                                             via Upwork   
4      via Higher Education Recruitment Consortium (H...   
...                                                  ...   
39709                              via My ArkLaMiss Jobs   
39710                              via My ArkLaMiss Jobs   
39711                              via My ArkLaMiss Jobs   
39712                              via My ArkLaMiss Jobs   
39713                              via My ArkLaMiss Jobs   

                                             description  \
0      As the leader in cloud-managed IT, Cisco Merak...   
1      As a Senior Business Analyst you will contribu...   
2      Overview:\n\nAmyx is seeking to hire a Data An...   
3      I am looking for someone to help me build an A...   
4      Position Vacancy – Data Analyst to support the...   
...                                                  ...   
39709  At Edward Jones, we help clients achieve their...   
39710  At Edward Jones, we help clients achieve their...   
39711  At Edward Jones, we help clients achieve their...   
39712  At Edward Jones, we help clients achieve their...   
39713  The Institutional Credit Management (ICM) grou...   

                                              extensions  \
0      ['22 hours ago', 'Work from home', 'Full-time'...   
1      ['53 minutes ago', 'Work from home', 'Full-tim...   
2                           ['3 hours ago', 'Full-time']   
3       ['23 hours ago', 'Work from home', 'Contractor']   
4                          ['20 hours ago', 'Full-time']   
...                                                  ...   
39709  ['23 hours ago', '76,798–130,764 a year', 'Ful...   
39710  ['23 hours ago', '106,916–182,047 a year', 'Fu...   
39711  ['23 hours ago', '106,916–182,047 a year', 'Fu...   
39712  ['23 hours ago', '106,916–182,047 a year', 'Fu...   
39713  ['24 hours ago', '105,850–158,780 a year', 'Fu...   

                                                  job_id  \
0      eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...   
1      eyJqb2JfdGl

In [5]:
jobs_data['description']

0        As the leader in cloud-managed IT, Cisco Merak...
1        As a Senior Business Analyst you will contribu...
2        Overview:\n\nAmyx is seeking to hire a Data An...
3        I am looking for someone to help me build an A...
4        Position Vacancy – Data Analyst to support the...
                               ...                        
39709    At Edward Jones, we help clients achieve their...
39710    At Edward Jones, we help clients achieve their...
39711    At Edward Jones, we help clients achieve their...
39712    At Edward Jones, we help clients achieve their...
39713    The Institutional Credit Management (ICM) grou...
Name: description, Length: 39714, dtype: object

In [6]:
!pip install --upgrade scipy

In [7]:
# # Delete files

# import os
# import shutil

# # Path to the Kaggle working directory
# kaggle_working_dir = "/kaggle/working"

# # Delete the files in the directory
# for filename in os.listdir(kaggle_working_dir):
#     file_path = os.path.join(kaggle_working_dir, filename)
#     try:
#         if os.path.isfile(file_path) or os.path.islink(file_path):
#             os.unlink(file_path)
#         elif os.path.isdir(file_path):
#             shutil.rmtree(file_path)
#     except Exception as e:
#         print('Failed to delete %s. Reason: %s' % (file_path, e))


In [8]:
import re
import zipfile
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [9]:
def setup_nltk_resources():
    """
    Sets up the custom NLTK data path and downloads necessary resources.
    Downloads 'wordnet' for lemmatization, 'stopwords' for stopwords removal, 
    and 'punkt' for sentence tokenization.
    """
    nltk_data_path = "/kaggle/working/nltk_data"
    nltk.data.path.append(nltk_data_path)
    
    nltk.download('wordnet', download_dir=nltk_data_path)
    nltk.download('stopwords', download_dir=nltk_data_path)
    nltk.download('punkt', download_dir=nltk_data_path)

In [10]:
def unzip_nltk_resource(zip_path, extract_to):
    """
    Unzips an NLTK resource file to a specified directory.

    Args:
    zip_path (str): The path to the zipped NLTK resource file.
    extract_to (str): The directory where the contents of the zip file will be extracted.
    """
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

In [11]:
def preprocess_text(text):
    """
    Preprocesses a given text string for NLP tasks. This includes cleaning the text,
    tokenizing, removing stopwords, and lemmatizing the words.

    Args:
    text (str): The text string to preprocess.

    Returns:
    str: The preprocessed text.
    """
    if not text:
        return ""
    text = re.sub(r'[\r\n\t]+', ' ', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    
    return ' '.join(lemmatized_text)

In [12]:
# Setup NLTK resources
setup_nltk_resources()

# Unzip wordnet
corpora_path = "/kaggle/working/nltk_data/corpora"
wordnet_zip = os.path.join(corpora_path, "wordnet.zip")
unzip_nltk_resource(wordnet_zip, corpora_path)

[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /kaggle/working/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
# Apply preprocessing
jobs_data['processed_description'] = jobs_data['description'].apply(preprocess_text)
jobs_data['processed_title'] = jobs_data['title'].apply(preprocess_text)

In [14]:
jobs_data['processed_description']

0        leader cloudmanaged cisco meraki connects pass...
1        senior business analyst contribute mission str...
2        overview amyx seeking hire data analystresearc...
3        looking someone help build ai tool pull data t...
4        position vacancy data analyst support ed cente...
                               ...                        
39709    edward jones help client achieve serious longt...
39710    edward jones help client achieve serious longt...
39711    edward jones help client achieve serious longt...
39712    edward jones help client achieve serious longt...
39713    institutional credit management icm group look...
Name: processed_description, Length: 39714, dtype: object

In [15]:
jobs_data['processed_title']

0                                             data analyst
1                 entry level business data analyst remote
2                                   data analystresearcher
3        data analystai expert help build website scrap...
4                                             data analyst
                               ...                        
39709                         marketing data bi analyst ii
39710                                     leaddata analyst
39711                                     leaddata analyst
39712                                     leaddata analyst
39713    institutional credit management lending data a...
Name: processed_title, Length: 39714, dtype: object

**Handle duplicates**

In [16]:
def drop_duplicates(df, column_name):
    """
    Drops duplicates based on a specified column from the DataFrame.

    Args:
    df (pd.DataFrame): The DataFrame from which to remove duplicates.
    column_name (str): The name of the column based on which duplicates will be identified.

    Returns:
    pd.DataFrame: DataFrame with duplicates removed based on the specified column.
    """
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    original_size = df.shape[0]
    df_cleaned = df.drop_duplicates(subset=[column_name])
    new_size = df_cleaned.shape[0]

    print(f"Dropped {original_size - new_size} duplicates from '{column_name}'. New dataset size: {new_size}")

    return df_cleaned

In [17]:
jobs_data_cleaned = drop_duplicates(jobs_data, column_name='description')

Dropped 11698 duplicates from 'description'. New dataset size: 28016


In [18]:
jobs_data_cleaned['processed_description']

0        leader cloudmanaged cisco meraki connects pass...
1        senior business analyst contribute mission str...
2        overview amyx seeking hire data analystresearc...
3        looking someone help build ai tool pull data t...
4        position vacancy data analyst support ed cente...
                               ...                        
39680    job summary business intelligence analyst iii ...
39681    current employee current employee ryder contra...
39703    build reporting dashboard tableau selfbuilt qu...
39705    lumen lumen guided belief humanity best techno...
39706    combine two fastestgrowing field planet cultur...
Name: processed_description, Length: 28016, dtype: object

In [19]:
jobs_data_cleaned['processed_title']

0                                             data analyst
1                 entry level business data analyst remote
2                                   data analystresearcher
3        data analystai expert help build website scrap...
4                                             data analyst
                               ...                        
39680                    business intelligence analyst iii
39681                             coop senior data analyst
39703                                         data analyst
39705                            lead fpa analyst remote u
39706                                  data analyst remote
Name: processed_title, Length: 28016, dtype: object

**Resume data**

In [20]:
resume_data = pd.read_csv('/kaggle/input/resume-dataset/UpdatedResumeDataSet.csv')

In [21]:
resume_data['Resume']

0      Skills * Programming Languages: Python (pandas...
1      Education Details \r\nMay 2013 to May 2017 B.E...
2      Areas of Interest Deep Learning, Control Syste...
3      Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4      Education Details \r\n MCA   YMCAUST,  Faridab...
                             ...                        
957    Computer Skills: â¢ Proficient in MS office (...
958    â Willingness to accept the challenges. â ...
959    PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...
960    COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...
961    Skill Set OS Windows XP/7/8/8.1/10 Database MY...
Name: Resume, Length: 962, dtype: object

In [22]:
# Apply preprocessing
resume_data['processed_resume'] = resume_data['Resume'].apply(preprocess_text)

In [23]:
resume_data['processed_resume']

0      skill programming language python panda numpy ...
1      education detail may may uitrgpv data scientis...
2      area interest deep learning control system des...
3      skill r python sap hana tableau sap hana sql s...
4      education detail mca ymcaust faridabad haryana...
                             ...                        
957    computer skill proficient m office word basic ...
958    willingness accept challenge positive thinking...
959    personal skill quick learner eagerness learn n...
960    computer skill software knowledge mspower poin...
961    skill set o window xp database mysql sql serve...
Name: processed_resume, Length: 962, dtype: object

In [24]:
resume_data_cleaned = drop_duplicates(resume_data, column_name='Resume')

Dropped 796 duplicates from 'Resume'. New dataset size: 166


In [25]:
resume_data_cleaned['processed_resume']

0      skill programming language python panda numpy ...
1      education detail may may uitrgpv data scientis...
2      area interest deep learning control system des...
3      skill r python sap hana tableau sap hana sql s...
4      education detail mca ymcaust faridabad haryana...
                             ...                        
894    computer skill proficient m office word basic ...
895    willingness accept challenge positive thinking...
896    personal skill quick learner eagerness learn n...
897    computer skill software knowledge mspower poin...
898    skill set o window xp database mysql sql serve...
Name: processed_resume, Length: 166, dtype: object

**Summary for Preprocessed Features for Resume and Job Descriptions**

1. resume_data_cleaned['processed_resume']
2. jobs_data_cleaned['processed_title']
3. jobs_data_cleaned['processed_description']


In [26]:
def add_token_count_column(df, column_name):
    """
    Adds a new column to the DataFrame with the token count for each entry in the specified column.
    This function creates a copy of the DataFrame to avoid 'SettingWithCopyWarning'.

    Args:
    df (pd.DataFrame): The DataFrame to process.
    column_name (str): The name of the column for which to count tokens.

    Returns:
    pd.DataFrame: DataFrame with an additional column 'token_count'.
    """
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    # Creating a copy of the DataFrame to avoid modifying a slice
    df_copy = df.copy()

    # Tokenize each entry in the specified column and count the number of tokens
    df_copy['token_count'] = df_copy[column_name].apply(lambda x: len(word_tokenize(x)) if pd.notnull(x) else 0)

    return df_copy


In [27]:
jobs_data_cleaned_with_tokens = add_token_count_column(jobs_data_cleaned, column_name='processed_description')


In [28]:
jobs_data_cleaned_with_tokens

Unnamed: 0  index                                              title  \
0               0      0                                       Data Analyst   
1               1      1       Entry Level - Business Data Analyst (Remote)   
2               2      2                            Data Analyst/Researcher   
3               3      3  Data analyst/AI expert to help build a website...   
4               4      4                                       Data Analyst   
...           ...    ...                                                ...   
39680       39680    253                  Business Intelligence Analyst III   
39681       39681    254                         COOP - Senior Data Analyst   
39703       39703    276                                       Data Analyst   
39705       39705    278                      Lead FP&A Analyst- Remote, US   
39706       39706    279                              Data Analyst - Remote   

                           company_name            location  \
0                          Cisco Meraki           Anywhere    
1                      Midwest Staffing           Anywhere    
2                            Amyx, Inc.    United States      
3                                Upwork           Anywhere    
4      Bank Street College of Education    United States      
...                                 ...                 ...   
39680                  Techdash Telecom       United States   
39681                      Ryder System   Oklahoma City, OK   
39703                    Advantage Tech   Overland Park, KS   
39705                             Lumen       United States   
39706                UnitedHealth Group       United States   

                                                     via  \
0                                           via LinkedIn   
1                                           via LinkedIn   
2                                             via Indeed   
3                                             via Upwork   
4      via Higher Education Recruitment Consortium (H...   
...                                                  ...   
39680                              via My ArkLaMiss Jobs   
39681                              via My ArkLaMiss Jobs   
39703                              via My ArkLaMiss Jobs   
39705                              via My ArkLaMiss Jobs   
39706                              via My ArkLaMiss Jobs   

                                             description  \
0      As the leader in cloud-managed IT, Cisco Merak...   
1      As a Senior Business Analyst you will contribu...   
2      Overview:\n\nAmyx is seeking to hire a Data An...   
3      I am looking for someone to help me build an A...   
4      Position Vacancy – Data Analyst to support the...   
...                                                  ...   
39680  Job Summary:\n\nThe Business Intelligence Anal...   
39681  Current Employees:\n\nIf you are a current emp...   
39703  What you will do:\n• Build reporting dashboard...   
39705  About Lumen\nLumen is guided by our belief tha...   
39706  Combine two of the fastest-growing fields on t...   

                                              extensions  \
0      ['22 hours ago', 'Work from home', 'Full-time'...   
1      ['53 minutes ago', 'Work from home', 'Full-tim...   
2                           ['3 hours ago', 'Full-time']   
3       ['23 hours ago', 'Work from home', 'Contractor']   
4                          ['20 hours ago', 'Full-time']   
...                                                  ...   
39680                      ['22 hours ago', 'Full-time']   
39681                      ['22 hours ago', 'Full-time']   
39703                      ['23 hours ago', 'Full-time']   
39705  ['23 hours ago', 'Full-time', 'Health insurance']   
39706  ['23 hours ago', 'Full-time', 'Health insurance']   

                                                  job_id  \
0      eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...   
1      eyJqb2JfdGl

In [29]:
resume_data_cleaned_with_tokens = add_token_count_column(resume_data_cleaned, column_name='processed_resume')


In [30]:
resume_data_cleaned_with_tokens

Category                                             Resume  \
0    Data Science  Skills * Programming Languages: Python (pandas...   
1    Data Science  Education Details \r\nMay 2013 to May 2017 B.E...   
2    Data Science  Areas of Interest Deep Learning, Control Syste...   
3    Data Science  Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...   
4    Data Science  Education Details \r\n MCA   YMCAUST,  Faridab...   
..            ...                                                ...   
894       Testing  Computer Skills: â¢ Proficient in MS office (...   
895       Testing  â Willingness to accept the challenges. â ...   
896       Testing  PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...   
897       Testing  COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...   
898       Testing  Skill Set OS Windows XP/7/8/8.1/10 Database MY...   

                                      processed_resume  token_count  
0    skill programming language python panda numpy ...          488  
1    education detail may may uitrgpv data scientis...          118  
2    area interest deep learning control system des...          180  
3    skill r python sap hana tableau sap hana sql s...          715  
4    education detail mca ymcaust faridabad haryana...           49  
..                                                 ...          ...  
894  computer skill proficient m office word basic ...          218  
895  willingness accept challenge positive thinking...          209  
896  personal skill quick learner eagerness learn n...          316  
897  computer skill software knowledge mspower poin...           87  
898  skill set o window xp database mysql sql serve...          202  

[166 rows x 4 columns]

**Dropping all unnecessary features**

In [31]:
# Dropping unnecessary columns from jobs data
jobs_data_final = jobs_data_cleaned_with_tokens[['processed_title', 'processed_description', 'token_count']]

# Dropping unnecessary columns from resume data
resume_data_final = resume_data_cleaned_with_tokens[['processed_resume', 'token_count']]


In [32]:
jobs_data_final.shape

(28016, 3)

In [33]:
resume_data_final.shape

(166, 2)

In [34]:
jobs_data_final

processed_title  \
0                                           data analyst   
1               entry level business data analyst remote   
2                                 data analystresearcher   
3      data analystai expert help build website scrap...   
4                                           data analyst   
...                                                  ...   
39680                  business intelligence analyst iii   
39681                           coop senior data analyst   
39703                                       data analyst   
39705                          lead fpa analyst remote u   
39706                                data analyst remote   

                                   processed_description  token_count  
0      leader cloudmanaged cisco meraki connects pass...          541  
1      senior business analyst contribute mission str...          217  
2      overview amyx seeking hire data analystresearc...          290  
3      looking someone help build ai tool pull data t...           25  
4      position vacancy data analyst support ed cente...          575  
...                                                  ...          ...  
39680  job summary business intelligence analyst iii ...          225  
39681  current employee current employee ryder contra...          516  
39703  build reporting dashboard tableau selfbuilt qu...          156  
39705  lumen lumen guided belief humanity best techno...          712  
39706  combine two fastestgrowing field planet cultur...          802  

[28016 rows x 3 columns]

In [35]:
resume_data_final

processed_resume  token_count
0    skill programming language python panda numpy ...          488
1    education detail may may uitrgpv data scientis...          118
2    area interest deep learning control system des...          180
3    skill r python sap hana tableau sap hana sql s...          715
4    education detail mca ymcaust faridabad haryana...           49
..                                                 ...          ...
894  computer skill proficient m office word basic ...          218
895  willingness accept challenge positive thinking...          209
896  personal skill quick learner eagerness learn n...          316
897  computer skill software knowledge mspower poin...           87
898  skill set o window xp database mysql sql serve...          202

[166 rows x 2 columns]

**Fearues**


In [36]:
jobs_data_final.info()


<class 'pandas.core.frame.DataFrame'>
Index: 28016 entries, 0 to 39706
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   processed_title        28016 non-null  object
 1   processed_description  28016 non-null  object
 2   token_count            28016 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 875.5+ KB


In [37]:
resume_data_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 166 entries, 0 to 898
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   processed_resume  166 non-null    object
 1   token_count       166 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.9+ KB


# Let's load Custom Summarization model

In [38]:
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

class TextSummarizer:
    """
    A text summarization class that uses a fine-tuned BART model to summarize text.

    Attributes:
        device (str): Device to run the model on, either 'cuda' or 'cpu'.
        model (BartForConditionalGeneration): The loaded BART model.
        tokenizer (BartTokenizer): The tokenizer for the BART model.
    """

    def __init__(self, model_name):
        """
        Initializes the TextSummarizer with a specified BART model.

        Args:
            model_name (str): The name or path of the fine-tuned BART model.
        """
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = BartForConditionalGeneration.from_pretrained(model_name).to(self.device)
        self.tokenizer = BartTokenizer.from_pretrained(model_name)

    def summarize(self, text, max_input_length=1024, max_output_length=150, min_output_length=40):
        """
        Summarizes the given text using the fine-tuned BART model.

        Args:
            text (str): The text to be summarized.
            max_input_length (int): The maximum length of the input text in tokens.
            max_output_length (int): The maximum length of the summary text in tokens.
            min_output_length (int): The minimum length of the summary text in tokens.

        Returns:
            str: The summarized text.
        """
        inputs = self.tokenizer([text], max_length=max_input_length, return_tensors='pt', truncation=True)
        summary_ids = self.model.generate(
            inputs['input_ids'].to(self.device),
            max_length=max_output_length,
            min_length=min_output_length,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True
        )
        return self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [39]:
summarizer = TextSummarizer("geekradius/bart-large-cnn-fintetuned-samsum-repo")


config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/328 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [40]:
!pip install tqdm


In [41]:
from tqdm import tqdm
import pandas as pd

def batch_summarize(df, text_col, summarizer, batch_size=10, output_col=None):
    """
    Summarizes text in batches.

    Args:
        df (pd.DataFrame): The DataFrame containing text to summarize.
        text_col (str): The column in the DataFrame with text to summarize.
        summarizer: The summarizer object or function.
        batch_size (int): The size of each batch for summarization.
        output_col (str, optional): The name of the output column for summarized text. 
                                   If None, defaults to text_col.

    Returns:
        pd.DataFrame: DataFrame with summarized text in the specified output column.
    """
    summarized_texts = []

    # Use the text_col as output_col if not specified
    if output_col is None:
        output_col = text_col

    # Iterate through the DataFrame in batches
    for start_idx in tqdm(range(0, len(df), batch_size), desc="Summarizing"):
        end_idx = start_idx + batch_size
        batch = df[text_col][start_idx:end_idx]
        
        # Summarize each batch
        summarized_batch = [summarizer.summarize(text) for text in batch]
        summarized_texts.extend(summarized_batch)

    # Create a new DataFrame with the summarized text
    return pd.DataFrame({output_col: summarized_texts})


# Summarize!

In [42]:
# Summarize the top 100 'processed_description' of jobs_data_final
top_jobs_data = jobs_data_final.head(100)

# Summariz jobs description
jobs_data_summarized = batch_summarize(top_jobs_data, 'processed_description', summarizer, batch_size=10, output_col='summarized_description')

# Summarize all 'processed_resume' in resume_data_final
resume_data_summarized = batch_summarize(resume_data_final, 'processed_resume', summarizer, batch_size=10, output_col='summarized_resume')


Summarizing: 100%|██████████| 17/17 [02:49<00:00,  9.99s/it]


In [43]:
resume_data_summarized.head(5)

summarized_resume
0  skill programming language python panda numpy ...
1  Education detail may may uitrGPv, data scienti...
2  area interest in deep learning control system ...
3  skill r python sap hana tableau, sana, hana, a...
4  Education detail mca ymcaust faridabadabadabad...

In [44]:
jobs_data_summarized.head(5)

summarized_description
0  Cisco is looking for a new employee to join it...
1  senior business analyst will work closely with...
2  Amyx is looking for a data analyst. The positi...
3  Someone will help build ai tool that can perfo...
4  The bank street education center is looking fo...

In [45]:
resume_data_summarized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166 entries, 0 to 165
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   summarized_resume  166 non-null    object
dtypes: object(1)
memory usage: 1.4+ KB


In [46]:
jobs_data_summarized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 1 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   summarized_description  100 non-null    object
dtypes: object(1)
memory usage: 928.0+ bytes


# Encoding with SentenceTransformers

In [47]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 855.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=73eb22e7427823c600b488d7000dab721049ca5cb46eb98bada4cb24ae029490
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [48]:
from sentence_transformers import SentenceTransformer
import pandas as pd

class SentenceTransformerEncoder:
    """
    A class to handle sentence encoding using Sentence Transformers, directly working with pandas DataFrames.
    This class encodes text data in a specified DataFrame column into vector representations.

    Attributes:
        model (SentenceTransformer): The Sentence Transformer model used for encoding.
    """

    def __init__(self, model_name='all-MiniLM-L6-v2'):
        """
        Initializes the SentenceTransformerEncoder with a specified Sentence Transformer model.

        Args:
            model_name (str): The name of the Sentence Transformer model.
        """
        self.model = SentenceTransformer(model_name)

    def encode_column(self, df, column, batch_size=32, encoded_column_suffix='_encoded'):
        """
        Encodes a specific column in a DataFrame and adds a new column with encoded vectors.

        Args:
            df (pd.DataFrame): The DataFrame containing the texts to encode.
            column (str): The name of the column to encode.
            batch_size (int): The size of each batch for processing.
            encoded_column_suffix (str): Suffix for the new column containing encoded vectors.

        Returns:
            pd.DataFrame: The original DataFrame with an additional column containing encoded vectors.
        
        Raises:
            ValueError: If the specified column is not found in the DataFrame.
        """
        if column not in df.columns:
            raise ValueError(f"Column '{column}' not found in DataFrame")
        
        # Encoding the text data in batches
        encoded_vectors = []
        for start_idx in range(0, len(df), batch_size):
            end_idx = min(start_idx + batch_size, len(df))
            batch_texts = df[column][start_idx:end_idx].tolist()
            batch_encoded = self.model.encode(batch_texts, show_progress_bar=True)
            encoded_vectors.extend(batch_encoded)

        # Adding the encoded vectors as a new column in the DataFrame
        df[column + encoded_column_suffix] = encoded_vectors
        return df

# # Example Usage
# encoder = SentenceTransformerEncoder()

# # Encoding the summarized job descriptions
# jobs_data_summarized_and_encoded = encoder.encode_column(jobs_data_summarized, 'summarized_description')

# # Encoding the summarized resumes
# resume_data_summarized_and_encoded = encoder.encode_column(resume_data_summarized, 'summarized_resume')


In [49]:
# Example Usage
encoder = SentenceTransformerEncoder(model_name='all-MiniLM-L6-v2')

# Encoding the summarized job descriptions
jobs_data_summarized_and_encoded = encoder.encode_column(jobs_data_summarized, 'summarized_description')

# Encoding the summarized resumes
resume_data_summarized_and_encoded = encoder.encode_column(resume_data_summarized, 'summarized_resume')


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [50]:
jobs_data_summarized_and_encoded

summarized_description  \
0   Cisco is looking for a new employee to join it...   
1   senior business analyst will work closely with...   
2   Amyx is looking for a data analyst. The positi...   
3   Someone will help build ai tool that can perfo...   
4   The bank street education center is looking fo...   
..                                                ...   
95  mi analyst trainee professional role requires ...   
96  Montefiore uba wecare is looking for a junior ...   
97  lumen lumen is a global technology leader digi...   
98  senior data analyst product data team zoom bri...   
99  Telus international aidata solution partner is...   

                       summarized_description_encoded  
0   [-0.073754296, -0.0085997805, 0.019644508, 0.0...  
1   [0.015300435, 0.020998282, -0.04914038, -0.050...  
2   [-0.0073945834, -0.041161846, 0.02017921, 0.02...  
3   [-0.06516003, 0.010788199, -0.07813146, -0.000...  
4   [-0.056208648, -0.022251448, -0.037455436, 0.0...  
..                                                ...  
95  [-0.043236256, -0.012699192, -0.061604585, 0.0...  
96  [-0.067706905, -0.014017106, -0.07185394, 0.06...  
97  [-0.04545598, -0.09242192, 0.019550018, 0.0015...  
98  [0.0059637628, -0.0017664266, 0.015467957, -0....  
99  [-0.052581355, -0.013363984, -0.028429616, -0....  

[100 rows x 2 columns]

In [51]:
resume_data_summarized_and_encoded

summarized_resume  \
0    skill programming language python panda numpy ...   
1    Education detail may may uitrGPv, data scienti...   
2    area interest in deep learning control system ...   
3    skill r python sap hana tableau, sana, hana, a...   
4    Education detail mca ymcaust faridabadabadabad...   
..                                                 ...   
161  Computer skill proficient m office word basic ...   
162  Dongare mandakini murlidhar is a student at a ...   
163  Dhanraj wagheducation detail is a company that...   
164  Computer skill software knowledge mspower poin...   
165  skill set o window xp database, css, jr testin...   

                             summarized_resume_encoded  
0    [-0.066532046, 0.018028306, -0.045124542, -0.0...  
1    [-0.035283532, -0.008672195, 0.025539452, 0.01...  
2    [-0.04613269, -0.005179984, -0.07708497, -0.05...  
3    [-0.02658479, -0.066101044, -0.014641958, 0.01...  
4    [-0.061161418, 0.0758177, -0.023499861, -0.033...  
..                                                 ...  
161  [-0.039111517, 0.054453403, 0.012831522, 0.034...  
162  [-0.07090609, 0.031574823, -3.1743195e-05, -0....  
163  [-0.059965115, 0.12965293, -0.04137429, 0.0416...  
164  [-0.05840614, 0.08813962, -0.012096944, -0.057...  
165  [-0.11341478, 0.03778131, -0.029018914, -0.026...  

[166 rows x 2 columns]

In [52]:
jobs_data_final

processed_title  \
0                                           data analyst   
1               entry level business data analyst remote   
2                                 data analystresearcher   
3      data analystai expert help build website scrap...   
4                                           data analyst   
...                                                  ...   
39680                  business intelligence analyst iii   
39681                           coop senior data analyst   
39703                                       data analyst   
39705                          lead fpa analyst remote u   
39706                                data analyst remote   

                                   processed_description  token_count  
0      leader cloudmanaged cisco meraki connects pass...          541  
1      senior business analyst contribute mission str...          217  
2      overview amyx seeking hire data analystresearc...          290  
3      looking someone help build ai tool pull data t...           25  
4      position vacancy data analyst support ed cente...          575  
...                                                  ...          ...  
39680  job summary business intelligence analyst iii ...          225  
39681  current employee current employee ryder contra...          516  
39703  build reporting dashboard tableau selfbuilt qu...          156  
39705  lumen lumen guided belief humanity best techno...          712  
39706  combine two fastestgrowing field planet cultur...          802  

[28016 rows x 3 columns]

In [53]:
resume_data_final

processed_resume  token_count
0    skill programming language python panda numpy ...          488
1    education detail may may uitrgpv data scientis...          118
2    area interest deep learning control system des...          180
3    skill r python sap hana tableau sap hana sql s...          715
4    education detail mca ymcaust faridabad haryana...           49
..                                                 ...          ...
894  computer skill proficient m office word basic ...          218
895  willingness accept challenge positive thinking...          209
896  personal skill quick learner eagerness learn n...          316
897  computer skill software knowledge mspower poin...           87
898  skill set o window xp database mysql sql serve...          202

[166 rows x 2 columns]

In [54]:
# Combine the jobs data
jobs_combined = pd.merge(
    jobs_data_final,
    jobs_data_summarized_and_encoded[['summarized_description', 'summarized_description_encoded']],
    left_index=True, right_index=True)

# Combine the resume data
resume_combined = pd.merge(
    resume_data_final,
    resume_data_summarized_and_encoded[['summarized_resume', 'summarized_resume_encoded']],
    left_index=True, right_index=True)

# Reset index of DataFrame
jobs_combined.reset_index(drop=True, inplace=True)
resume_combined.reset_index(drop=True, inplace=True)

In [55]:
jobs_combined.head(5)

processed_title  \
0                                       data analyst   
1           entry level business data analyst remote   
2                             data analystresearcher   
3  data analystai expert help build website scrap...   
4                                       data analyst   

                               processed_description  token_count  \
0  leader cloudmanaged cisco meraki connects pass...          541   
1  senior business analyst contribute mission str...          217   
2  overview amyx seeking hire data analystresearc...          290   
3  looking someone help build ai tool pull data t...           25   
4  position vacancy data analyst support ed cente...          575   

                              summarized_description  \
0  Cisco is looking for a new employee to join it...   
1  senior business analyst will work closely with...   
2  Amyx is looking for a data analyst. The positi...   
3  Someone will help build ai tool that can perfo...   
4  The bank street education center is looking fo...   

                      summarized_description_encoded  
0  [-0.073754296, -0.0085997805, 0.019644508, 0.0...  
1  [0.015300435, 0.020998282, -0.04914038, -0.050...  
2  [-0.0073945834, -0.041161846, 0.02017921, 0.02...  
3  [-0.06516003, 0.010788199, -0.07813146, -0.000...  
4  [-0.056208648, -0.022251448, -0.037455436, 0.0...

In [56]:
resume_combined.head(5)

processed_resume  token_count  \
0  skill programming language python panda numpy ...          488   
1  education detail may may uitrgpv data scientis...          118   
2  area interest deep learning control system des...          180   
3  skill r python sap hana tableau sap hana sql s...          715   
4  education detail mca ymcaust faridabad haryana...           49   

                                   summarized_resume  \
0  skill programming language python panda numpy ...   
1  Education detail may may uitrGPv, data scienti...   
2  area interest in deep learning control system ...   
3  skill r python sap hana tableau, sana, hana, a...   
4  Education detail mca ymcaust faridabadabadabad...   

                           summarized_resume_encoded  
0  [-0.066532046, 0.018028306, -0.045124542, -0.0...  
1  [-0.035283532, -0.008672195, 0.025539452, 0.01...  
2  [-0.04613269, -0.005179984, -0.07708497, -0.05...  
3  [-0.02658479, -0.066101044, -0.014641958, 0.01...  
4  [-0.061161418, 0.0758177, -0.023499861, -0.033...

# Upload to Vector Databae - Qdrant

In [57]:
!pip install qdrant-client


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for qdrant-client from https://files.pythonhosted.org/packages/80/43/cf47f8c8c231612ab3f92f5ddcbd78b07bfca3e31656a9b87d200d545cb2/qdrant_client-1.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for grpcio-tools>=1.41.0 from https://files.pythonhosted.org/packages/3c/7d/00a156dba65c9965e6e94988ab518c4ea88f95e1b70c2b61b34dd65124b5/grpcio_tools-1.60.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for httpx[http2]>=0.14.0 from https://files.pythonhosted.org/packages/39/9b/4937d841aee9c2c8102d9a4eeb800c7dad25386caabb4a1bf5010df81a57/httpx-0.26.0-py3-none-any.whl.metadata
  Obtaining dependency information for portalocker<3.0.0,>=2.7.0 from https://files.pythonhosted.org/packages/17/9e/87671efcca80ba6203811540ed1f9c0462c1609d2281d7b7f53cef05da3d/portalocker-2.8.2-py3-none-any.whl.metadata
  Obtaining dependency information for protobuf<5.0dev,>=4.21.6 from https://files.pythonhosted.o

In [58]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance, Record, Filter
from random import uniform


class QdrantInterface:
    """
    A class for interfacing with the Qdrant vector database.

    Attributes:
        client (QdrantClient): Client instance for interacting with Qdrant.
        vector_dimension (int): Dimension of the vectors used in the collection.
    """

    """
    A class for interfacing with the Qdrant vector database.
    ...
    """
    def __init__(self, url, api_key, vector_dimension):
        """
        Initializes the QdrantInterface with the specified Qdrant URL, API key, and vector dimension.

        Args:
            url (str): Full URL of the Qdrant server.
            api_key (str): API key for Qdrant.
            vector_dimension (int): Dimension of vectors to be stored in Qdrant.
        """
        self.client = QdrantClient(url=url, api_key=api_key)
        self.vector_dimension = vector_dimension

    def create_collection(self, collection_name, distance_metric=Distance.COSINE):
        """
        Creates or recreates a collection in Qdrant.

        Args:
            collection_name (str): Name of the collection.
            distance_metric (Distance): Distance metric for vector comparisons.
        """
        self.client.recreate_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=self.vector_dimension, distance=distance_metric)
        )

    def save_to_qdrant(self, df, collection_name, vector_col, payload_cols, batch_size=100):
        """
        Saves a DataFrame to Qdrant in batches.

        Args:
            df (pd.DataFrame): DataFrame containing data to save.
            collection_name (str): Name of the collection in Qdrant.
            vector_col (str): Name of the column containing vectors.
            payload_cols (list[str]): List of column names to include as payload.
            batch_size (int): Number of records to process in each batch.
        """

        for start_idx in range(0, len(df), batch_size):
            end_idx = min(start_idx + batch_size, len(df))
            batch = df.iloc[start_idx:end_idx]
            records = []
            for idx, row in batch.iterrows():
                # Debug print
                print(f"Index: {idx}, Vector Type: {type(row[vector_col])}, First 10 Elements: {row[vector_col][:10]}")
                record = Record(
                    id=idx,
                    vector=row[vector_col],
                    payload={col: row[col] for col in payload_cols}
                )
                records.append(record)
            self.client.upload_records(collection_name=collection_name, records=records)

            
            
    def retrieve_specific_records(self, collection_name, ids):
        """
        Retrieves specific records by their IDs from a Qdrant collection.

        Args:
            collection_name (str): The name of the collection.
            ids (list): List of record IDs to retrieve.

        Returns:
            List of specific records from the collection.
        """
        return self.client.retrieve(collection_name=collection_name, ids=ids)
    
    def view_sample_records(self, collection_name, vector_dimension, limit=10):
        """
        Retrieves a sample of records from a Qdrant collection using a dummy search.

        Args:
            collection_name (str): The name of the collection.
            vector_dimension (int): Dimension of vectors in the collection.
            limit (int): The number of records to retrieve.

        Returns:
            List of sample records from the collection.
        """
        # Generate a random vector
        random_vector = [uniform(-1, 1) for _ in range(vector_dimension)]

        # Perform a dummy search
        return self.client.search(
            collection_name=collection_name,
            query_vector=random_vector,
            limit=limit
        )
    
    def match_resumes_to_jobs(self, resume_vector, top_k=10):
        """
        Matches a given resume vector to job postings.

        Args:
            resume_vector (list): The vector representation of a resume.
            top_k (int): Number of top similar matches to return.

        Returns:
            List of matched job postings with similarity scores.
        """
        hits = self.client.search(
            collection_name="jobs",
            query_vector=resume_vector,
            limit=top_k,
            with_payload=True
        )
        return [(hit.payload, hit.score) for hit in hits]

    def match_jobs_to_resumes(self, job_vector, top_k=10):
        """
        Matches a given job vector to resumes.

        Args:
            job_vector (list): The vector representation of a job posting.
            top_k (int): Number of top similar matches to return.

        Returns:
            List of tuples containing matched resumes and their similarity scores.
        """
        hits = self.client.search(
            collection_name="resumes",
            query_vector=job_vector,
            limit=top_k,
            with_payload=True
        )
        return [(hit.payload, hit.score) for hit in hits]



In [59]:
vector_dimension = encoder.model.get_sentence_embedding_dimension()


In [60]:
vector_dimension

384

In [61]:
QUADRANT_ENDPOINT = 'https://b9f78ca1-e97e-467b-8f8f-e0246f3324da.us-east4-0.gcp.cloud.qdrant.io:6333'
QUADRANT_API_KEY = '_yUt9VmEJr7iM0dDPDvgFhqrhT97GGeDMxNMvsKQvJGNe3n_YUrhCg'

vector_dimension = encoder.model.get_sentence_embedding_dimension()
qdrant_interface = QdrantInterface(QUADRANT_ENDPOINT, QUADRANT_API_KEY, vector_dimension)
qdrant_interface.create_collection('jobs', Distance.COSINE)
qdrant_interface.create_collection('resumes', Distance.COSINE)

# Function to ensure vectors are in list format
def ensure_list_format(df, vector_col):
    df[vector_col] = df[vector_col].apply(lambda x: x.tolist() if hasattr(x, 'tolist') else x)
    return df

# Ensure vectors are in the correct format before uploading
jobs_combined = ensure_list_format(jobs_combined, 'summarized_description_encoded')
resume_combined = ensure_list_format(resume_combined, 'summarized_resume_encoded')



In [62]:
sample_job_vector = jobs_combined['summarized_description_encoded'].iloc[0]
print(type(sample_job_vector))
print(sample_job_vector[:10])  # Print first 10 elements of the vector


<class 'list'>
[-0.07375429570674896, -0.00859978049993515, 0.01964450813829899, 0.09265688061714172, -0.06824183464050293, -0.0785854235291481, -0.045186035335063934, 0.00362506159581244, -0.07954569160938263, -0.03503796458244324]


In [63]:
# Now upload to Qdrant
qdrant_interface.save_to_qdrant(jobs_combined, 'jobs', 'summarized_description_encoded', ['processed_title', 'processed_description', 'token_count', 'summarized_description'])
qdrant_interface.save_to_qdrant(resume_combined, 'resumes', 'summarized_resume_encoded', ['processed_resume', 'token_count', 'summarized_resume'])


Index: 0, Vector Type: <class 'list'>, First 10 Elements: [-0.07375429570674896, -0.00859978049993515, 0.01964450813829899, 0.09265688061714172, -0.06824183464050293, -0.0785854235291481, -0.045186035335063934, 0.00362506159581244, -0.07954569160938263, -0.03503796458244324]
Index: 1, Vector Type: <class 'list'>, First 10 Elements: [0.01530043501406908, 0.020998282358050346, -0.04914037883281708, -0.05081146955490112, -0.09512876719236374, -0.021102063357830048, -0.039611414074897766, 0.019752545282244682, -0.04912000522017479, -0.051214974373579025]
Index: 2, Vector Type: <class 'list'>, First 10 Elements: [-0.0073945834301412106, -0.04116184636950493, 0.020179210230708122, 0.02629982680082321, -0.08550816029310226, -0.06205545738339424, 0.033380184322595596, -0.08494385331869125, -0.012823820114135742, -0.04381182789802551]
Index: 3, Vector Type: <class 'list'>, First 10 Elements: [-0.06516002863645554, 0.010788198560476303, -0.07813145965337753, -0.0007555446936748922, -0.0252292063

In [64]:
# View sample records from the 'jobs' collection
sample_jobs_records = qdrant_interface.view_sample_records('jobs', vector_dimension, limit=2)

In [65]:
sample_jobs_records

[ScoredPoint(id=4, version=2, score=0.083088115, payload={'processed_description': 'position vacancy data analyst support ed center project portfolio bank street education center ed center summary position bank street education center ed center division within bank street college education college created support college systemslevel work school district currently seeking data analyst support project across ed center portfolio lead measurement planning well develop associated data collection tool support assigned project key relationship data analyst report director impact work closely project team well broader sspp team responsibility collaboration supervisor project team broader sspp team data analyst responsible following analytics collaborating relevant team member draft measurement plan clearly document desired outcome goal well associated project metric collaboration key stakeholder creating data collection tool capture data aligned key project metric defined project lead measure

In [66]:
# Retrieve specific records by IDs from the 'jobs' collection
specific_jobs_records = qdrant_interface.retrieve_specific_records('jobs', ids=[1])


In [67]:
specific_jobs_records

[Record(id=1, payload={'processed_description': 'senior business analyst contribute mission strategically supporting requirement collection process mapping project management successful ontime delivery across multiple project report directly manager enterprise pmo work closely department head across company youll contribute role expect create businessfunctional requirement document process map asis tobe state focus enterprise scalability leveraging automation build forecast model adhoc analysis support capital allocation decision conduct interview focus group issue survey support business process analysis ass business performance across multiple attribute cost quality service compliance critically analyze business situation partner process owner develop solution reengineer existing business process support automation multiple manual business process identify support resolution escalate report problem issue work closely department leader team internal customer highlight factor affect wo

# Matching:

With the encoded representations of job descriptions and resumes, we can then proceed to match them based on their similarity.

This step typically involves calculating similarity scores between resume encodings and job description encodings, which can be achieved through methods like cosine similarity.


In [68]:
# Example job vector from the jobs_combined DataFrame
example_job_vector = jobs_combined['summarized_description_encoded'].iloc[44]

In [69]:
jobs_combined['summarized_description'][44]

"Data analyst visualization engineer is a full-time position in information technology. The position requires a minimum clearance and requires regular percentage travel. It's an information technology job with a focus on technology and data analysis."

In [70]:
# Find top 5 matching resumes for the example job
matched_resumes = qdrant_interface.match_jobs_to_resumes(example_job_vector, top_k=5)
for resume, score in matched_resumes:
    print(f"Matched Resume: {resume['summarized_resume']}, Score: {score}")


Matched Resume: Expertise data quantitative analysis, predictive modeling, data visualization, business intelligence, data mining, machine learning, data analysis and data analysis. Exercises of aegis school data science business and business analytics., Score: 0.45388216
Matched Resume: skill python tableau data visualization r studio machine learning statistic iabac certified data scientist with experience year managing business data science consulting leading innovation project bringing business idea working real world solution., Score: 0.43393213
Matched Resume: Education details for January include bachelor engineering, master's degree in computer engineering, and a position as a consultant for a computer security project in Mumbai. The job responsibility is to create a user adaptable new environment for the project., Score: 0.37413222
Matched Resume: Education detail may may uitrGPv, data scientist data scientist matelabs skill detail python exprience le year month statsmodels, a